In [1]:
import pandas as pd
import re 
from bs4 import BeautifulSoup
import pymorphy2
import pymystem3
from collections import Counter
import numpy as np
mystem=pymystem3.Mystem()

In [2]:
df_train = pd.read_csv('train.csv', sep='\t', index_col='id')


In [3]:
def mystem_func(text):
    return ''.join(mystem.lemmatize(text))

In [28]:
def data_preproc(df_input):
    df_output = df_input.copy()
    df_output.loc[:, 'information'] = df_output['name']+' '+df_output['description']
    df_output.drop(axis=1, columns=['name', 'description'])
    df_output.information.map(BeautifulSoup('information', 'lxml'))
    df_output.information.map(mystem_func)
    return df_output

In [12]:
df_preproc = df_train.pipe(data_preproc)

In [13]:
!telegram-send 'success'

In [8]:
df_preproc.drop(axis=1, columns=['description', 'name'])

,target,information
id,,
0,1,Заведующий отделом/секцией в магазин YORK (Уру...
1,0,Наладчик станков и манипуляторов с ПУ Обязанно...
2,0,Разработчик С++ (Криптограф) <strong>Требовани...
3,0,Фрезеровщик <p>Условия:</p> <ul> <li>На работу...
4,1,Мерчендайзер/продавец-консультант <p><strong>К...
5,0,Мастер по эксплуатации зданий <p><strong>Обяза...
6,1,Торговый представитель <em><strong>НА СТАБИЛЬН...
7,1,Торговый представитель <strong>Обязанности:</s...
8,1,Менеджер по продажам <p><strong>Обязанности:</...


In [15]:
#from catboost import Pool, CatBoostRegressor, CatBoostClassifier
#import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint as randint

model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,1),
                              max_df=0.835)),
    ('clf', LogisticRegression(random_state=13)),
])
cv = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)

In [1]:
X = df_train_preproc.iloc[:, :-1].values
y = df_train_preproc.iloc[:, -1].values

NameError: name 'df_train_preproc' is not defined

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

In [18]:
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.835, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...alty='l2', random_state=13, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [19]:
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

0.9898438781393942

In [29]:
df_test = pd.read_csv('test.csv', sep='\t', index_col='id')
df_test_preproc = df_test.pipe(data_preproc)

In [30]:
!telegram-send 'success'

In [32]:
df_test_preproc.drop(axis=1, columns=['name', 'description'])


,information
id,
200000,Дизайнер-консультант мебели <p><strong>Обязанн...
200001,Продавец-консультант (ТЦ на Пушкина) <p><stron...
200002,Менеджер по продажам <p>Торговый Дом «Форт» эт...
200003,Продавец-консультант в магазин одежды (ТЦ Волн...
200004,Специалист по охране труда <strong>Обязанности...
200005,Эксперт по обеспечению качества при сооружении...
200006,Торговый представитель (Арзамас) <p><strong>Об...
200007,Заместитель генерального директора по производ...
200008,Backend Rust developer <p><strong>Storiqa </st...


In [33]:
submission = pd.DataFrame()
submission.loc[:,'id'] = test.reset_index()['id']
submission.loc[:,'target'] = model.predict_proba(df_test_preproc['information'])[:, 1]
submission.to_csv('submission.csv', sep=',', index=False, encoding='utf-8')

In [34]:
submission

,id,target
0,200000,0.985377
1,200001,0.999285
2,200002,0.993330
3,200003,0.994317
4,200004,0.012532
5,200005,0.065059
6,200006,0.988288
7,200007,0.155402
8,200008,0.039511
9,200009,0.252776
